In [1]:
import fastbook
fastbook.setup_book()

from fastbook import *
import re
from IPython.display import display,HTML

from fastai.text.all import *

In [2]:
train = pd.read_csv('datasets/Train.csv')
test = pd.read_csv('datasets/Test.csv')
train.head(1)

,id,content,category
0,SW0,"SERIKALI imesema haitakuwa tayari kuona amani na utulivu wa nchi inachezewa huku ikisisitiza uwepo wa umoja kati ya wananchi bila kujali tofauti ya imani, kabila au itikadi yoyote.Hayo yalisemwa na Naibu Waziri wa Mambo ya Ndani ya Nchi, Hamad Yussuf Masauni wakati akifungua semina ya siku mbili iliyofanyika jijini Dar es Salaam ikiwahusisha viongozi wa taasisi za Kiislamu, lengo ikiwa ni kuwakumbusha kuhubiri amani katika sehemu zao.Naibu Waziri amesema mwelekeo na malengo ya Serikali ya Awamu ya Tano ni kukuza maendeleo katika sehemu mbalimbali nchini lengo ikiwa kuinua maisha ya wananc...",Kitaifa


In [3]:
train.columns,test.columns

(Index(['id', 'content', 'category'], dtype='object'),
 Index(['swahili_id', 'content'], dtype='object'))

### Read the entire texts

In [540]:
train.iloc[2,3]

' SERIKALI imetoa miezi sita kwa taasisi zote za umma ambazo hazitumii mfumo wa GePG katika ukusanyaji wa fedha kufanya hivyo na baada ya hapo itafanya ukaguzi na kuwawajibisha maofi sa masuhuli walioshindwa kutekeleza hilo.Akifungua mkutano wa mwaka wa kwanza wa watumiaji wa mfumo huo jana jijini hapa, Naibu Katibu Mkuu Wizara ya Fedha na Mipango, Dk Hartibu Kazungu alisema ukaguzi huo ambao utaanza kufanyika Juni mwakani baada ya kipindi kilichowekwa kupita ili kuwabaini maofisa wazembe.Alisema, wizara yake itafanya ukaguzi wa ofisi zote za umma ili kubaini kama kuna fedha zinazokusanywa bila kupitia kwenye mfumo wa GePG na kuwa ofisi itakayobainika kutotumia mfumo huo maofisa masuhuli watawajibishwa kwa mujibu wa sheria. Mnifikishie ujumbe wa kuzikumbusha taasisi ambazo hazijaanza kutumia mfumo wa ukusanyaji mapato wa GePG kufanya hivyo, kwani tunataka hadi kufikia Juni 30, mwaka 2019, kusiwe na fedha yoyote kwenye taasisi ya umma ambayo inakusanywa nje ya mfumo huu,  alisema.Hata h

In [198]:
def clean_row(x):
    x = ''.join(x.split('\'')).strip('[]')
    return re.sub(r"[^\x00-\x7F]", ' ', x)
    
txts = L(
    list(train['content'].apply(clean_row).values) + 
#     list(train.loc[:,'content'].values) + 
#     list(test.loc[:,'content'].values) 
    list(test['content'].apply(clean_row).values)
)
len(txts)

6439

### Concatenate into one big stream

In [343]:
spacy = WordTokenizer()
tkn = Tokenizer(spacy)

toks200 = txts[:200].map(tkn)

num = Numericalize()
num.setup(toks200)

nums200 = toks200.map(num)

dl = LMDataLoader(nums200)

In [347]:
x,y = first(dl)
x.shape, y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [348]:
TextBlock.from_df??

In [424]:
train['content_cleaned'] = train['content'].apply(clean_row)
test['content_cleaned'] = test['content'].apply(clean_row)

train.to_csv('train/Train.csv')
test.to_csv('test/Test.csv')

In [449]:
for i in train.index:
    filename = f'train/{train.loc[i,"id"]}.txt'
    with open(filename,'w') as f:
        f.write(train.loc[i,'content_cleaned'])

In [452]:
for i in test.index:
    filename = f'test/{test.loc[i,"swahili_id"]}.txt'
    with open(filename,'w') as f:
        f.write(test.loc[i,'content_cleaned'])

In [381]:
get_files('',extensions=['.csv'],folders=['train','test',],recurse=True)

(#2) [Path('test/Test.csv'),Path('train/Train.csv')]

In [372]:
get_text_files??

In [364]:
partial??

In [406]:
TextBlock??

In [407]:
TextBlock.from_df('gh',is_lm=True)

In [398]:
train[:4].to_csv('train/Train.csv')

In [455]:
path = Path('')
path

Path('.')

In [458]:
get_news = partial(get_text_files, folders=['train','test'])

db = DataBlock(
    blocks = TextBlock.from_folder(path,is_lm=True),
    get_items = get_news,
    splitter = RandomSplitter(0.1),
)

dls_lm = db.dataloaders(path, path=path, bs=128, seq_len=80)

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `n_workers` has to be changed to 0 to avoid getting stuck


In [461]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj wamewahakikishia kuwa hivi sasa ushiriki wao unahitajika kwa maendeleo ya kiuchumi na kwa faida ya wananchi wa nchi hizo.rais xxmaj jakaya xxmaj kikwete alisema sekta ya biashara , wako tayari kufanya kazi sehemu xxunk kupata fedha . xxmaj alisema huu ni wakati wa watendaji katika nchi hizo , kufanya kazi kibiashara na kuachana na tabia ya urasimu , ambayo imekuwa xxunk wawekezaji . xxmaj kama tutafanya kazi kibiashara , miradi ya miundombinu itaweza kukamilika katika kipindi cha muda","xxmaj wamewahakikishia kuwa hivi sasa ushiriki wao unahitajika kwa maendeleo ya kiuchumi na kwa faida ya wananchi wa nchi hizo.rais xxmaj jakaya xxmaj kikwete alisema sekta ya biashara , wako tayari kufanya kazi sehemu xxunk kupata fedha . xxmaj alisema huu ni wakati wa watendaji katika nchi hizo , kufanya kazi kibiashara na kuachana na tabia ya urasimu , ambayo imekuwa xxunk wawekezaji . xxmaj kama tutafanya kazi kibiashara , miradi ya miundombinu itaweza kukamilika katika kipindi cha muda mfupi"
1,"lengo letu ni moja kuijenga nchi hii , alisema xxmaj mchange . xxmaj aidha alisema endapo xxmaj muswada huo xxunk na xxmaj rais na kuwa sheria , mapendekezo yake kwa serikali ni kwamba xxunk kubadili vipengele vilivyomo ndani , bali inapaswa iende mbali na kuwachukulia hatua watu wote watakaokwenda kinyume na sheria husika . xxbos xxup wakazi wa xxmaj mkoa wa xxmaj kagera hususani xxmaj bukoba xxmaj mjini pamoja na wilaya za xxmaj misenyi , xxmaj kyerwa na xxmaj karagwe","letu ni moja kuijenga nchi hii , alisema xxmaj mchange . xxmaj aidha alisema endapo xxmaj muswada huo xxunk na xxmaj rais na kuwa sheria , mapendekezo yake kwa serikali ni kwamba xxunk kubadili vipengele vilivyomo ndani , bali inapaswa iende mbali na kuwachukulia hatua watu wote watakaokwenda kinyume na sheria husika . xxbos xxup wakazi wa xxmaj mkoa wa xxmaj kagera hususani xxmaj bukoba xxmaj mjini pamoja na wilaya za xxmaj misenyi , xxmaj kyerwa na xxmaj karagwe ,"


In [648]:
class MyLLM10(Module):
    def __init__(self, vocab_sz, n_hidden,n_layers,p):
        self.i_h = nn.Embedding(vocab_sz,n_hidden)
        self.rnn = nn.LSTM(n_hidden,n_hidden, n_layers, batch_first=True)
        self.drop = nn.Dropout(p) # Dropout
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        self.h_o.weight = self.i_h.weight # Weight Tying
        self.h = [torch.zeros(n_layers, bs, n_hidden) for _ in range(2)] # Some changes here
    
    def forward(self,x):
        res,h = self.rnn(self.i_h(x), self.h)
        out = self.drop(res)
        self.h = [h_.detach() for h_ in h]
        return self.h_o(out),res,out
    
    def reset(self):
        for h in self.h:
            h.zero_ # Some changes here
    

class MyLLM11(Module):
    "AWD-LSTM inspired by https://arxiv.org/abs/1708.02182"
    initrange=0.1

    def __init__(self, 
        vocab_sz:int, # Size of the vocabulary
        emb_sz:int, # Size of embedding vector
        n_hid:int, # Number of features in hidden state
        n_layers:int, # Number of LSTM layers
        pad_token:int=1, # Padding token id
        hidden_p:float=0.2, # Dropout probability for hidden state between layers
        input_p:float=0.6, # Dropout probability for LSTM stack input
        embed_p:float=0.1, # Embedding layer dropout probabillity
        weight_p:float=0.5, # Hidden-to-hidden wight dropout probability for LSTM layers
        bidir:bool=False # If set to `True` uses bidirectional LSTM layers
    ):
        store_attr('emb_sz,n_hid,n_layers,pad_token')
        self.bs = 1
        self.n_dir = 2 if bidir else 1
        self.encoder = nn.Embedding(vocab_sz, emb_sz, padding_idx=pad_token)
        self.encoder_dp = EmbeddingDropout(self.encoder, embed_p)
        self.rnns = nn.ModuleList([self._one_rnn(emb_sz if l == 0 else n_hid, (n_hid if l != n_layers - 1 else emb_sz)//self.n_dir,
                                                 bidir, weight_p, l) for l in range(n_layers)])
        self.encoder.weight.data.uniform_(-self.initrange, self.initrange)
        self.input_dp = RNNDropout(input_p)
        self.hidden_dps = nn.ModuleList([RNNDropout(hidden_p) for l in range(n_layers)])
        self.reset()

    def forward(self, inp:Tensor, from_embeds:bool=False):
        bs,sl = inp.shape[:2] if from_embeds else inp.shape
        if bs!=self.bs: self._change_hidden(bs)

        output = self.input_dp(inp if from_embeds else self.encoder_dp(inp))
        new_hidden = []
        for l, (rnn,hid_dp) in enumerate(zip(self.rnns, self.hidden_dps)):
            output, new_h = rnn(output, self.hidden[l])
            new_hidden.append(new_h)
            if l != self.n_layers - 1: output = hid_dp(output)
        self.hidden = to_detach(new_hidden, cpu=False, gather=False)
        return tensor(np.zeros((10240,1))),tensor(np.zeros((10240,1))),tensor(np.zeros((10240)))

    def _change_hidden(self, bs):
        self.hidden = [self._change_one_hidden(l, bs) for l in range(self.n_layers)]
        self.bs = bs

    def _one_rnn(self, n_in, n_out, bidir, weight_p, l):
        "Return one of the inner rnn"
        rnn = nn.LSTM(n_in, n_out, 1, batch_first=True, bidirectional=bidir)
        return WeightDropout(rnn, weight_p)

    def _one_hidden(self, l):
        "Return one hidden state"
        nh = (self.n_hid if l != self.n_layers - 1 else self.emb_sz) // self.n_dir
        return (one_param(self).new_zeros(self.n_dir, self.bs, nh), one_param(self).new_zeros(self.n_dir, self.bs, nh))

    def _change_one_hidden(self, l, bs):
        if self.bs < bs:
            nh = (self.n_hid if l != self.n_layers - 1 else self.emb_sz) // self.n_dir
            return tuple(torch.cat([h, h.new_zeros(self.n_dir, bs-self.bs, nh)], dim=1) for h in self.hidden[l])
        if self.bs > bs: return (self.hidden[l][0][:,:bs].contiguous(), self.hidden[l][1][:,:bs].contiguous())
        return self.hidden[l]

    def reset(self):
        "Reset the hidden states"
        [r.reset() for r in self.rnns if hasattr(r, 'reset')]
        self.hidden = [self._one_hidden(l) for l in range(self.n_layers)]
        
class MyLLM4(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz,n_hidden)
        self.h_h = nn.Linear(n_hidden,n_hidden)
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        self.h = 0
    
    def forward(self,x):
        num_cols = x.shape[1]
        outs = []
        for i in range(num_cols):
            self.h = self.h + self.i_h(x[:,i]) # Access the Emb vector for the respective column
            self.h = self.h_h(self.h) # Connect to a Linear NN
            self.h = F.relu(self.h) # Our Non - liearization
            outs.append(self.h_o(self.h)) # Pass to the output layer
        res = self.h
        self.h = self.h.detach()
        return torch.stack(outs, dim=1),res,outs
    
    def reset(self):
        self.h = 0

In [649]:
tensor(np.zeros((100000)))

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [650]:
len(dls_lm.vocab)

28480

In [651]:
MyLSTMModel = MyLLM11(len(dls_lm.vocab),64,64,4)
# MyLSTMModel = MyLLM4(len(dls_lm.vocab),64)
# MyLSTMModel = MyLLM10(len(dls_lm.vocab),64,4,0.4)

In [658]:
# learn = LMLearner(
#     dls_lm, MyLSTMModel, 
#     loss_func=CrossEntropyLossFlat(),
#     metrics=[accuracy, Perplexity()]
# )

learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

KeyboardInterrupt: 

In [ ]:
learn.fit_one_cycle(1, 2e-2)

In [ ]:
learn.fit_one_cycle(1, 2e-2)

In [545]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.760979,4.770746,0.267244,118.007217,26:32


In [546]:
learn.fit_one_cycle(2, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.638702,4.681375,0.276487,107.918358,25:55
1,4.426968,4.558434,0.286272,95.433914,26:28


In [468]:
learn = language_model_learner(
    dls_lm, MyLLM10, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

KeyError: <class '__main__.MyLLM10'>

In [297]:
text = ' xbos '.join([l.strip() for l in txts[:200]])
text[:100]

'SERIKALI imesema haitakuwa tayari kuona amani na utulivu wa nchi inachezewa huku ikisisitiza uwepo w'

In [299]:
## Tokenize

sp = SubwordTokenizer(vocab_sz=2000)
# sp.setup(txts[:1])
sp.setup(txts)

# spacy = WordTokenizer()
# tkn = Tokenizer(spacy)

# tokens = tkn(text)
# tokens

{'sp_model': Path('tmp/spm.model')}

In [306]:
tokens = list(sp([text]))[0]

In [307]:
## To Numericalize


# num = Numericalize()
# num.setup(tokens)

# # Generate the vocab with unique tokens
vocab = L(*tokens).unique()

word2idx = {w:i for i, w in enumerate(vocab)}
nums = L(word2idx[w] for w in tokens)
nums

(#112814) [0,1,2,3,4,5,6,7,8,9...]

In [308]:
len(nums),len(vocab)

(112814, 1977)

## Build a language model

### Predict each word based on the previous 3

In [309]:
L((tokens[i:i+3],tokens[i+3]) for i in range(0,len(tokens)-4,3))

(#37604) [(['▁SER', 'IKA', 'LI'], '▁ime'),(['▁ime', 'sema', '▁hai'], 'ta'),(['ta', 'kuwa', '▁tayari'], '▁kuona'),(['▁kuona', '▁amani', '▁na'], '▁u'),(['▁u', 'tu', 'li'], 'vu'),(['vu', '▁wa', '▁nchi'], '▁ina'),(['▁ina', 'che', 'ze'], 'wa'),(['wa', '▁huku', '▁iki'], 'sisitiza'),(['sisitiza', '▁uwepo', '▁wa'], '▁umoja'),(['▁umoja', '▁kati', '▁ya'], '▁wananchi')...]

The model can't use the above but the numericalized version of the above

In [310]:
seqs = L((tensor(nums[i:i+3]),nums[i+3]) for i in range(0,len(nums)-4,3))
seqs

(#37604) [(tensor([0, 1, 2]), 3),(tensor([3, 4, 5]), 6),(tensor([6, 7, 8]), 9),(tensor([ 9, 10, 11]), 12),(tensor([12, 13, 14]), 15),(tensor([15, 16, 17]), 18),(tensor([18, 19, 20]), 21),(tensor([21, 22, 23]), 24),(tensor([24, 25, 16]), 26),(tensor([26, 27, 28]), 29)...]

### Creating the DataLoader with a `batch_size` of 64

In [311]:
bs = 64
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(seqs[:cut],seqs[cut:],shuffle=False,bs=bs)

In [312]:
x,y = dls.one_batch()
x.shape,y.shape

(torch.Size([64, 3]), torch.Size([64]))

## Our Language Model in PyTorch

In [313]:
class MyLLM1(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz,n_hidden)
        self.h_h = nn.Linear(n_hidden,n_hidden)
        self.h_o = nn.Linear(n_hidden,vocab_sz)
    
    def forward(self,x):
        h = self.i_h(x[:,0]) # Access the Emb vector for the first column
        h = self.h_h(h) # Connect to the first set of Linear NN
        h = F.relu(h) # Our Non - liearization
        
        h = h + self.i_h(x[:,1]) # Add the Emb vector for the second column
        h = self.h_h(h) # Pass to the Linear Network
        h = F.relu(h) # Again the Non-linearization
        
        h = h + self.i_h(x[:,2]) # Add the Emb vector for the third column
        h = self.h_h(h) # Pass to the Linear Network
        h = F.relu(h) # Again the Non-linearization
        
        h = self.h_o(h) # Pass to the output layer
        return h

Rewriting it as

In [314]:
class MyLLM2(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz,n_hidden)
        self.h_h = nn.Linear(n_hidden,n_hidden)
        self.h_o = nn.Linear(n_hidden,vocab_sz)
    
    def forward(self,x):
        h = 0
        num_cols = x.shape[1]
        for i in range(num_cols):
            h = h + self.i_h(x[:,i]) # Access the Emb vector for the respective column
            h = self.h_h(h) # Connect to a Linear NN
            h = F.relu(h) # Our Non - liearization
        h = self.h_o(h) # Pass to the output layer
        return h

### Training

In [315]:
# Baseline model of always predicting the mode of the tokens

n,counts = 0,torch.zeros(len(vocab))
for x,y in dls.valid:
    n += y.shape[0]
    for i in range_of(vocab): counts[i] += (y==i).long().sum()
idx = torch.argmax(counts)
idx, vocab[idx.item()], counts[idx].item()/n

(tensor(35), ',', 0.03297433851881399)

In [316]:
learn = Learner(dls, MyLLM2(len(vocab), 1024), loss_func=F.cross_entropy, 
                metrics=accuracy)
learn.fit_one_cycle(4, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,5.797340,5.822598,0.092142,00:57
1,4.115510,5.921351,0.111022,00:58
2,2.717089,6.425860,0.127643,00:56
3,2.376848,6.855087,0.131100,00:54


Slightly better

### Maintaining the state of the RNN

In [317]:
class MyLLM3(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz,n_hidden)
        self.h_h = nn.Linear(n_hidden,n_hidden)
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        self.h = 0
    
    def forward(self,x):
        num_cols = x.shape[1]
        for i in range(num_cols):
            self.h = self.h + self.i_h(x[:,i]) # Access the Emb vector for the respective column
            self.h = self.h_h(self.h) # Connect to a Linear NN
            self.h = F.relu(self.h) # Our Non - liearization
        out = self.h_o(self.h) # Pass to the output layer
        
        self.h = self.h.detach()
        return out
    
    def reset(self):
        self.h = 0

In [318]:
m = len(seqs)//bs
m,bs,len(seqs)

(587, 64, 37604)

In [319]:
def group_chunks(ds, bs):
    m = len(ds) // bs
    new_ds = L()
    for i in range(m): new_ds += L(ds[i + m*j] for j in range(bs))
    return new_ds

In [320]:
list(group_chunks(seqs[:20],6))#.reshape(4,-1)

[(tensor([0, 1, 2]), 3),
 (tensor([ 9, 10, 11]), 12),
 (tensor([18, 19, 20]), 21),
 (tensor([26, 27, 28]), 29),
 (tensor([28, 34, 35]), 36),
 (tensor([42, 43, 44]), 45),
 (tensor([3, 4, 5]), 6),
 (tensor([12, 13, 14]), 15),
 (tensor([21, 22, 23]), 24),
 (tensor([29, 30, 31]), 32),
 (tensor([36, 37, 38]), 39),
 (tensor([45, 46, 47]), 48),
 (tensor([6, 7, 8]), 9),
 (tensor([15, 16, 17]), 18),
 (tensor([24, 25, 16]), 26),
 (tensor([32, 14, 33]), 28),
 (tensor([39, 40, 41]), 42),
 (tensor([48, 21, 11]), 49)]

In [321]:
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(
    group_chunks(seqs[:cut], bs), 
    group_chunks(seqs[cut:], bs), 
    bs=bs, drop_last=True, shuffle=False)

### Training

In [24]:
learn = Learner(dls, MyLLM3(len(vocab), 1024), loss_func=F.cross_entropy, 
                metrics=accuracy,cbs=ModelResetter)
learn.fit_one_cycle(10, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,5.910364,6.097558,0.172414,02:09
1,4.002608,6.482233,0.191092,02:06
2,2.467951,7.328877,0.176904,02:06
3,1.519905,8.706957,0.170797,02:09
4,0.959000,10.245851,0.169540,02:06
5,0.610137,10.246753,0.168103,02:07
6,0.438787,10.063964,0.180316,02:07
7,0.312056,10.027181,0.188398,02:06
8,0.263799,9.995111,0.191092,02:07
9,0.336758,9.742608,0.195582,02:07


### Creating more signal

In [322]:
sl = 16
seqs = L((tensor(nums[i:i+sl]), tensor(nums[i+1:i+sl+1]))
         for i in range(0,len(nums)-sl-1,sl))
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(group_chunks(seqs[:cut], bs),
                             group_chunks(seqs[cut:], bs),
                             bs=bs, drop_last=True, shuffle=False)

Looking at the first element of `seqs`, we can see that it contains two lists of the same size. The second list is the same as the first, but offset by one element:

In [323]:
class MyLLM4(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz,n_hidden)
        self.h_h = nn.Linear(n_hidden,n_hidden)
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        self.h = 0
    
    def forward(self,x):
        num_cols = x.shape[1]
        outs = []
        for i in range(num_cols):
            self.h = self.h + self.i_h(x[:,i]) # Access the Emb vector for the respective column
            self.h = self.h_h(self.h) # Connect to a Linear NN
            self.h = F.relu(self.h) # Our Non - liearization
            outs.append(self.h_o(self.h)) # Pass to the output layer
        
        self.h = self.h.detach()
        return torch.stack(outs, dim=1)
    
    def reset(self):
        self.h = 0

In [324]:
def loss_func(inp, targ):
    return F.cross_entropy(inp.view(-1, len(vocab)), targ.view(-1))

In [325]:
learn = Learner(dls, MyLLM4(len(vocab), 1024), loss_func=loss_func, 
                metrics=accuracy,cbs=ModelResetter)
learn.fit_one_cycle(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,6.220513,5.796507,0.094638,00:44
1,5.071121,5.185163,0.136586,00:46
2,4.194140,5.053366,0.156294,00:42
3,3.569609,5.116390,0.164728,00:44
4,3.181340,5.093949,0.169389,00:43


In [326]:
def sample_pred(length):
    to_pred = nums[95:95+length]
    ans = nums[95+1:95+length+1]
    
    result = learn.predict(tensor([to_pred]))[0].argmax(1)
    print(result,ans)

    prompt = ""
    for i in range(len(to_pred)):
        prompt += vocab[to_pred[i]] +' '
#     print(prompt)
    prompt += " <>  "

    for i in range(len(result)):
        prompt += vocab[result[i]] + ' '
#     return prompt
    print(prompt)
    
    prompt = ' '.join(prompt.split(' ')[:len(ans)])
    prompt += "  <>  "

    for i in range(len(ans)):
        prompt += vocab[ans[i]] + ' '
    print(prompt)

In [327]:
sample_pred(10)

tensor([ 15, 194, 340,  85,  86,  38,  87, 113,  61, 222]) [82, 83, 84, 85, 86, 38, 87, 35, 88, 79]
▁wa ▁taasisi ▁za ▁Ki i s la mu , ▁lengo  <>  vu ▁hiyo ▁umma i s la mu ▁nchini ▁ ▁la 
▁wa ▁taasisi ▁za ▁Ki i s la mu , ▁lengo  <>  ▁taasisi ▁za ▁Ki i s la mu , ▁lengo ▁ikiwa 


I am actually very surprised. Like I am wowed already.

The original text is " of the Government of the Fifth Phase is to promote "<br>
The predicted text is " of Tanzania's Fifth Phase under promotion "

<b>NB</b>: 
<li> Translation done via Google Translate </li>
<li> This prediction was done on the training set (But it is still remarkable I believe) </li>

### Multi-Layer RNNs

In [328]:
class MyLLM5(Module):
    def __init__(self, vocab_sz, n_hidden,n_layers):
        self.i_h = nn.Embedding(vocab_sz,n_hidden)
        
        self.network = []
        for i in range(n_layers):
            self.network.append(nn.Linear(n_hidden,n_hidden))
            self.network.append(nn.ReLU())
        self.h_h = nn.Sequential(
            *self.network[:-1]
        )
#         self.h_h = nn.Linear(n_hidden,n_hidden)
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        self.h = 0
    
    def forward(self,x):
        num_cols = x.shape[1]
        outs = []
        for i in range(num_cols):
            self.h = self.h + self.i_h(x[:,i]) # Access the Emb vector for the respective column
            self.h = self.h_h(self.h) # Connect to a Linear NN
            self.h = F.relu(self.h) # Our Non - liearization
            outs.append(self.h_o(self.h)) # Pass to the output layer
        
        self.h = self.h.detach()
        return torch.stack(outs, dim=1)
    
    def reset(self):
        self.h = 0

In [32]:
learn = Learner(dls, MyLLM5(len(vocab), 1024,4), loss_func=loss_func, 
                metrics=accuracy,cbs=ModelResetter)
learn.fit_one_cycle(15, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,7.832971,6.765987,0.044739,02:25
1,6.556630,6.272907,0.160278,02:19
2,5.952004,6.013952,0.190002,02:20
3,5.511847,5.976256,0.211121,02:21
4,5.108160,6.024221,0.216064,02:21
5,4.775147,6.203149,0.220398,02:20
6,4.493296,6.327516,0.227173,02:21
7,4.273300,6.438957,0.224670,02:18
8,4.130563,6.620975,0.222656,02:19
9,4.093142,6.345071,0.230530,02:18


In other words

In [329]:
class MyLLM6(Module):
    def __init__(self, vocab_sz, n_hidden,n_layers):
        self.i_h = nn.Embedding(vocab_sz,n_hidden)
        self.rnn = nn.RNN(n_hidden,n_hidden, n_layers, batch_first=True)
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        self.h = torch.zeros(n_layers, bs, n_hidden) # Some changes here
    
    def forward(self,x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(res)
    
    def reset(self):
        self.h.zero_ # Some changes here

In [151]:
learn = Learner(dls, MyLLM6(len(vocab), 64,8), loss_func=loss_func, 
                metrics=accuracy,cbs=ModelResetter)
learn.fit_one_cycle(40, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,9.124417,9.022078,0.122498,00:14
1,8.783082,8.455340,0.122498,00:16
2,8.109900,7.619846,0.122498,00:16
3,7.321177,6.941107,0.122498,00:16
4,6.744375,6.657060,0.122498,00:16
5,6.491624,6.652512,0.122498,00:16
6,6.415926,6.688025,0.122498,00:17
7,6.396910,6.725154,0.122498,00:17
8,6.393860,6.759254,0.122498,00:17
9,6.394257,6.788418,0.122498,00:16


In [34]:
learn = Learner(dls, MyLLM6(len(vocab), 1024,4), loss_func=CrossEntropyLossFlat(), # Change in the loss function
                metrics=accuracy,cbs=ModelResetter)
learn.fit_one_cycle(15, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,7.200701,6.618497,0.123840,01:48
1,6.362325,6.229457,0.166504,01:48
2,5.823647,6.021464,0.202515,01:50
3,5.363237,6.079515,0.210083,01:49
4,4.953303,6.177130,0.211670,01:47
5,4.613482,6.241570,0.214600,01:55
6,4.299594,6.321890,0.216125,01:53
7,4.032951,6.331525,0.215576,01:47
8,3.806790,6.365344,0.218506,01:51
9,3.682621,6.379865,0.219299,01:46


Due to the problems of exploding or diasappearing activations, We will consider the use of LSTMs

### LSTM

In [330]:
class MyLLM7(Module):
    def __init__(self, ni, h):
        self.forget_gate = nn.Linear(ni + nh, nh)
        self.input_gate = nn.Linear(ni + nh, nh)
        self.cell_gate = nn.Linear(ni + nh, nh)
        self.output_gate = nn.Linear(ni + nh, nh)
    
    def forward(self, x, state):
        h,c = state
        h = torch.cat([h, x], dim=1)
        forget = torch.sigmoid(self.forget_gate(h))
        c = c * forget
        
        inputer = torchinput_gateigmoid(self.input_gate(h)) * torch.tanh(self.cell_gate(h))
        c = c + inputer
        
        outputer = torch.sigmoid(self.output_gate(h))
        h = torch.tanh(c) * outputer
        return h, (h,c)

Refactoring the above for increase in GPU performance

In [331]:
class MyLLM8(Module):
    def __init__(self, ni, h):
        self.ih = nn.Linear(ni,4*nh)
        self.hh = nn.Linear(nh,4*nh)
    
    def forward(self, x, state):
        h,c = state
        
        gates = (self.ih(x) + self.hh(h)).chunk(4,1)
        ingate, forgetgate, outgate = map(torch.sigmoid, gates[:3]) ## Note: order does not matter here
        cellgate = torch.tanh(gates[3])

        c = forgetgate*c + ingate*cellgate
        h = outgate * torch.tanh(c)
        return h, (h,c)

Now the LTSM network

In [332]:
class MyLLM9(Module):
    def __init__(self, vocab_sz, n_hidden,n_layers):
        self.i_h = nn.Embedding(vocab_sz,n_hidden)
        self.rnn = nn.LSTM(n_hidden,n_hidden, n_layers, batch_first=True)
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        self.h = [torch.zeros(n_layers, bs, n_hidden) for _ in range(2)] # Some changes here
    
    def forward(self,x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = [h_.detach() for h_ in h]
        return self.h_o(res)
    
    def reset(self):
        for h in self.h:
            h.zero_ # Some changes here

In [38]:
learn = Learner(dls, MyLLM9(len(vocab), 1024, 4), 
                loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(15, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,7.978433,6.681934,0.122498,03:42
1,6.850881,6.777476,0.122498,03:44
2,6.598370,6.803113,0.122498,03:51
3,6.429319,6.598124,0.122498,03:42
4,6.079883,6.307435,0.150940,03:41
5,5.721493,6.203966,0.170410,03:51
6,5.413800,6.209413,0.174133,04:04
7,5.146466,6.146118,0.186951,03:42
8,4.899539,6.264096,0.185486,03:35
9,4.717854,6.252806,0.184509,03:34


### Regularizing
We will add the following methods as seen in the paper taught by Jeremy;
<li> Dropout </li>
<li> Activation Regularization (AR) and Temporal Activation Regularization (TAR) </li>
<li> Weight Tying </li>

The LSTM that uses these techniques are called AWD-LSTM according to the author.

In [333]:
class MyLLM10(Module):
    def __init__(self, vocab_sz, n_hidden,n_layers,p):
        self.i_h = nn.Embedding(vocab_sz,n_hidden)
        self.rnn = nn.LSTM(n_hidden,n_hidden, n_layers, batch_first=True)
        self.drop = nn.Dropout(p) # Dropout
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        self.h_o.weight = self.i_h.weight # Weight Tying
        self.h = [torch.zeros(n_layers, bs, n_hidden) for _ in range(2)] # Some changes here
    
    def forward(self,x):
        res,h = self.rnn(self.i_h(x), self.h)
        out = self.drop(res)
        self.h = [h_.detach() for h_ in h]
        return self.h_o(out),res,out
    
    def reset(self):
        for h in self.h:
            h.zero_ # Some changes here

In [334]:
learn = Learner(dls, MyLLM10(len(vocab), 1024, 4, 0.5), 
                loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy, 
                cbs=[ModelResetter,RNNRegularizer(alpha=2,beta=1)]
               )

# OR
learn = TextLearner(dls, MyLLM10(len(vocab), 1024, 4, 0.4),
                   loss_func = CrossEntropyLossFlat(), metrics=accuracy)

learn.fit_one_cycle(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,6.744831,6.390353,0.052823,04:09



KeyboardInterrupt



In [42]:
learn = Learner(dls, MyLLM10(len(vocab), 1024, 4, 0.5), 
                loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy, 
                cbs=[ModelResetter,RNNRegularizer(alpha=2,beta=1)]
               )

# OR
learn = TextLearner(dls, MyLLM10(len(vocab), 1024, 4, 0.4),
                   loss_func = CrossEntropyLossFlat(), metrics=accuracy)

learn.fit_one_cycle(15, 1e-3, wd=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,7.567925,7.175681,0.122925,03:09
1,7.111494,6.670097,0.150391,03:20
2,6.544221,6.191212,0.207458,03:26
3,5.991594,6.060913,0.219971,03:29
4,5.410181,6.122361,0.226318,03:29
5,4.774519,6.354394,0.225830,03:17


KeyboardInterrupt: 

In [61]:
tensor([58]*64).dim()

1

In [92]:
len(tokens), len(nums)

(83865, 83865)

In [93]:
L(tokens[:12])

(#12) ['xbos','xupp','serikali','imesema','haitakuwa','tayari','kuona','amani','na','utulivu'...]

In [335]:
to_p_nums = nums[-994:-970]
to_p_nums = list(dls.valid.dataset[0][0])# + list(dls.valid.dataset[1][0])

In [336]:
to_p = tensor([to_p_nums[:-1]]*64)

prediction = learn.predict(to_p)#,[58]*64)])
for p in prediction[0]:
    print(vocab[p],end=' ')

▁na ▁ s na ▁wa na na na ▁ya . . . ▁ya ▁ya ▁ya 

In [337]:
for a in to_p_nums[1:]:
    print(vocab[a],end= ' ')

▁A th u ma n ▁Ki hamia ▁alisema ▁hayo ▁aki wasilisha ▁ma da ▁ya ▁U 